<a href="https://colab.research.google.com/github/MatthewYancey/Capitol_Faces/blob/main/src/facial_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Embeddings
This notebook loops through the all videos, finds the distinct faces in each video and creates embeddings/encodings for the faces.

In [ ]:
!pip install face_recognition
!pip install ffmpeg-python

In [ ]:
import os
import glob
import sys
import pandas as pd
import cv2
from tqdm import tnrange, tqdm_notebook
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/repos/Capitol_Faces/src')
import helpers.general as gen
import helpers.detection_and_clustering as dc

In [3]:
# parameters
data_path = '/content/gdrive/MyDrive/repos/Capitol_Faces/data/'
face_min_size = 80
clustering_threshold = 0.5
get_vid_length = False

## Generates the list of vidoes to process

In [ ]:
# gets the selected videos
top_n = 100
df_meta = pd.read_csv(data_path + 'df_jan6attack_individuals.csv')
df_meta = df_meta.loc[df_meta['date']=='2021-01-06', :]

# gets the top n commonly sighted individuals in photos
vid_list = df_meta.groupby('id')['video'].count().reset_index()
vid_list = vid_list.sort_values('video', ascending=False).reset_index(drop=True)
vid_list = vid_list.loc[:top_n, 'id'].to_list()
vid_list = df_meta.loc[df_meta['id'].isin(vid_list), 'video'].to_list()
vid_list = list(set(vid_list))
vid_list = [os.path.basename(v[:-1]) for v in vid_list]
print(f'Number of videos: {len(vid_list)}')

## Loads metadata

In [ ]:
# Loads the video data frame and the faces data frame
df_video_meta = pd.read_csv(data_path + 'df_video_meta.csv')

# checks if we have the videos we need to process
missing_videos = [v for v in vid_list if v not in df_video_meta['vid_id'].to_list()]
print(f'Number of missing videos: {len(missing_videos)}')

print(f'Number of videos already processed: {df_video_meta.loc[df_video_meta["vid_id"].isin(vid_list) & df_video_meta["face_detection"] == 1, :].shape[0]}')

# vid_list = df_video_meta.loc[df_video_meta["face_detection"] == 0, 'vid_id'].to_list()
vid_list = df_video_meta.loc[(df_video_meta["vid_id"].isin(vid_list)) & (df_video_meta["face_detection"] == 0), 'vid_id'].to_list()
print(f'Number of videos to be processed: {len(vid_list)}')

# how many minutes of video do we have to process?
if get_vid_length:
    gen.get_video_summary(vid_list, data_path)

In [ ]:
df_faces = pd.read_csv(data_path + 'df_faces.csv')
column_names = df_faces.columns

# loops through the vids and gets the unique face
for i in tnrange(len(vid_list)):
    
    print(f'Finding faces in {vid_list[i]}')
    cap = cv2.VideoCapture(data_path + 'video/' + vid_list[i] + '.mp4')    
    if cap.isOpened():

        # gets all the faces and encodings from the video
        vid_faces = dc.detect_faces(cap, vid_list[i], data_path, min_size=face_min_size)

        # if we found any faces
        if len(vid_faces) > 0:
            # clusters the faces
            df_faces_vid = pd.DataFrame(vid_faces, columns=['vid_id', 'frame', 'position'] + [f'encoding_{e}' for e in list(range(128))])
            df_faces_vid = dc.cluster_faces(df_faces_vid, clustering_threshold)

            # get the faces at the center of each cluster
            df_faces_vid = dc.get_cluster_center(df_faces_vid)

            # add the new faces to the face dataframe
            df_faces_vid.drop('cluster', axis=1, inplace=True)
            next_id = max(df_faces['id']) + 1
            df_faces_vid['id'] = range(next_id, next_id + df_faces_vid.shape[0])
            df_faces_vid = df_faces_vid.loc[:, ['id'] + list(df_faces_vid.columns[:-1])]
            df_faces = df_faces.append(df_faces_vid)
            df_faces.to_csv(data_path + 'df_faces.csv', index=False)

        # updates the metadata that this video has been processed
        df_video_meta.loc[df_video_meta['vid_id'] == vid_list[i], 'face_detection'] = 1
        df_video_meta.to_csv(data_path + 'df_video_meta.csv', index=False)

    else:
        print(f'Bad path to video {vid_list[i]}')